# Arguments

In [1]:
num_of_epochs = 100
batch_size = 32
dataset_path_plantvillage = '../datasets/plantvillage'
dataset_path_self = '../datasets/self'
dataset_path_validation = '../datasets/validation'

dataset_paths = [dataset_path_plantvillage, dataset_path_self, dataset_path_validation]

warmup_model_path = 'model-warmup.h5'
model_save_path = 'model-final.h5'
checkpoint_path = 'checkpoints-finetuning.hdf5'

input_width = 224
input_height = 224
input_depth = 3


# Select training classes

In [2]:
class_paths_training = ['../datasets/plantvillage/___Late_blight', '../datasets/plantvillage/___Target_Spot', '../datasets/plantvillage/___healthy', '../datasets/plantvillage/___Mosaic_Virus', '../datasets/plantvillage/___Yellow_Leaf_Curl_Virus', '../datasets/plantvillage/___Leaf_Mold', '../datasets/plantvillage/___Bacterial_spot', '../datasets/self/___Early_blight', '../datasets/self/___Appids', '../datasets/self/___Leaf_miner']

# Load dataset

In [3]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.preprocessing.image   import ImageDataGenerator
from keras.optimizers import RMSprop
from keras.layers import Input
from keras.models import Model
from keras .applications import VGG16
from keras.optimizers import SGD
from keras.models import Model
from imutils import paths
import numpy as np
import os

import sys
sys.path.append('..')

from utils.preprocessors.resize_image_preprocessor import resizeImagePreprocessor
from utils.preprocessors.image_to_array_preprocessor import ImageToArrayPreprocessor
from utils.io.simple_dataset_loader import SimpleDatasetLoader

# construct the image generator for data augmentation
aug = ImageDataGenerator(   rotation_range=30,
                            width_shift_range=0.1,
                            height_shift_range=0.1,
                            shear_range=0.2,
                            zoom_range=0.2,
                            horizontal_flip=True,
                            fill_mode='nearest')

# Load image paths
image_paths = []
print("[INFO] loading images...")
for path in class_paths_training:
    image_paths.extend(list(paths.list_images(path)))

# Get unique classnames
class_names = [pt.split(os.path.sep)[-2] for pt in image_paths]
class_names = [str(x) for x in np.unique(class_names)]

# Initial image preprocessing
aap = resizeImagePreprocessor(input_width, input_height)
iap= ImageToArrayPreprocessor()

#Load image data and perform image data preprocessing
sdl = SimpleDatasetLoader(preprocessors=[aap,iap])
(data,labels)  = sdl.load(image_paths,verbose=500)
data = data.astype("float") / 255.0


# train test split
(train_x,test_x,train_y,test_y) = train_test_split(data,labels,test_size=0.25,random_state=42)

# convert the labels from integers to vectors
train_y = LabelBinarizer().fit_transform(train_y)
test_y = LabelBinarizer().fit_transform(test_y)



Using TensorFlow backend.


[INFO] loading images...
[INFO]: Processed 500/18087
[INFO]: Processed 1000/18087
[INFO]: Processed 1500/18087
[INFO]: Processed 2000/18087
[INFO]: Processed 2500/18087
[INFO]: Processed 3000/18087
[INFO]: Processed 3500/18087
[INFO]: Processed 4000/18087
[INFO]: Processed 4500/18087
[INFO]: Processed 5000/18087
[INFO]: Processed 5500/18087
[INFO]: Processed 6000/18087
[INFO]: Processed 6500/18087
[INFO]: Processed 7000/18087
[INFO]: Processed 7500/18087
[INFO]: Processed 8000/18087
[INFO]: Processed 8500/18087
[INFO]: Processed 9000/18087
[INFO]: Processed 9500/18087
[INFO]: Processed 10000/18087
[INFO]: Processed 10500/18087
[INFO]: Processed 11000/18087
[INFO]: Processed 11500/18087
[INFO]: Processed 12000/18087
[INFO]: Processed 12500/18087
[INFO]: Processed 13000/18087
[INFO]: Processed 13500/18087
[INFO]: Processed 14000/18087
[INFO]: Processed 14500/18087
[INFO]: Processed 15000/18087
[INFO]: Processed 15500/18087
[INFO]: Processed 16000/18087
[INFO]: Processed 16500/18087
[INFO

# Load model from disk

In [4]:
# import tensorflow as tf
# print(tf.__version__)


# model = tf.keras.models.load_model(
#     warmup_model_path,
#     custom_objects=None,
#     compile=False
# )

from keras.models import load_model

model = load_model(warmup_model_path)

# Print index of layers

In [5]:
# iterate for all layers in the network and print its' index value
for (i,layer) in enumerate(model.layers):
    print("[INFO] {}\t{}".format(i,layer.__class__.__name__))

[INFO] 0	InputLayer
[INFO] 1	Conv2D
[INFO] 2	Conv2D
[INFO] 3	MaxPooling2D
[INFO] 4	Conv2D
[INFO] 5	Conv2D
[INFO] 6	MaxPooling2D
[INFO] 7	Conv2D
[INFO] 8	Conv2D
[INFO] 9	Conv2D
[INFO] 10	MaxPooling2D
[INFO] 11	Conv2D
[INFO] 12	Conv2D
[INFO] 13	Conv2D
[INFO] 14	MaxPooling2D
[INFO] 15	Conv2D
[INFO] 16	Conv2D
[INFO] 17	Conv2D
[INFO] 18	MaxPooling2D
[INFO] 19	Flatten
[INFO] 20	Dense
[INFO] 21	Dropout
[INFO] 22	Dense


# Unfreeze final CONV layers

In [6]:
for layer in model.layers[11:]:
    layer.trainable = True
print('unfrozen')

unfrozen


# Compile model

In [7]:
from keras.optimizers import SGD

# Build the model from the new
print("[INFO] re-compiling model ...")
opt = SGD(lr=0.001, momentum=0.09)
# Fine-tuning with a small learning rate
model.compile(loss = 'categorical_crossentropy',optimizer = opt,
              metrics=['accuracy'])


[INFO] re-compiling model ...


# Checkpoints

In [8]:
from keras.callbacks import ModelCheckpoint


checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_loss', mode='min', 
save_best_only=True, verbose=1)

callbacks = [checkpoint]



In [9]:
#load checkpoints if existing

import os

epochs_done = 0
num_of_epochs = 10

if(os.path.exists(checkpoint_path)):
    model.load_weights(checkpoint_path)
    num_of_epochs = num_of_epochs - epochs_done


# Training Monitor

In [10]:
from utils.callbacks.training_monitor import TrainingMonitor
import pathlib
import json
import os

fig_path = "plot"
json_path = "values.json"
values_path = 'values.json'

start_epoch = 127

callbacks.append(TrainingMonitor(fig_path, json_path, start_epoch))

# Fit model

In [11]:
num_of_epochs = 2

H = model.fit_generator(
    aug.flow(train_x,train_y, batch_size = 32),
             validation_data = (test_x,test_y),
             epochs=num_of_epochs,
             steps_per_epoch = len(train_x) //32,
             verbose = 1,
             callbacks=callbacks)

Epoch 1/2
423/423 [==============================] - 229s 540ms/step - loss: 0.0087 - accuracy: 0.9968 - val_loss: 0.0177 - val_accuracy: 0.9947

Epoch 00001: val_loss improved from inf to 0.01771, saving model to checkpoints-finetuning.hdf5
Epoch 2/2
423/423 [==============================] - 216s 511ms/step - loss: 0.0087 - accuracy: 0.9975 - val_loss: 0.0226 - val_accuracy: 0.9929

Epoch 00002: val_loss did not improve from 0.01771


# Save model

In [12]:
model.save(model_save_path)

# Load model form disk

In [13]:

# from keras.models import load_model

# model_save_path = 'model-final.h5'
# model = load_model(model_save_path)

# Evaluate

In [14]:
from sklearn.metrics import classification_report

print("[INFO] evaluating after initialization...")
predictions = model.predict(test_x,batch_size=batch_size)

print(classification_report(test_y.argmax(axis =1),
                            predictions.argmax(axis =1),
                            target_names=class_names, 
                            digits=4))

[INFO] evaluating after initialization...
                           precision    recall  f1-score   support

                ___Appids     1.0000    1.0000    1.0000       259
        ___Bacterial_spot     0.9962    0.9981    0.9972       531
          ___Early_blight     0.9973    0.9786    0.9878       373
           ___Late_blight     0.9935    0.9978    0.9957       459
             ___Leaf_Mold     0.9917    0.9917    0.9917       242
            ___Leaf_miner     0.9868    0.9978    0.9923       451
          ___Mosaic_Virus     0.8707    1.0000    0.9309       101
           ___Target_Spot     0.9916    1.0000    0.9958       356
___Yellow_Leaf_Curl_Virus     1.0000    0.9860    0.9929      1357
               ___healthy     1.0000    1.0000    1.0000       393

              avg / total     0.9934    0.9929    0.9930      4522

